In [ ]:
import h5py
import torch
import os
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm.auto import tqdm
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# --- 1. Global Configurations and Device Allocation ---
# Utilizing CUDA if available for accelerated deep learning performance
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BASE_PATH = '/kaggle/input/brain-to-text-25/t15_copyTask_neuralData/hdf5_data_final'

# Defining vocabulary: Using character-level tokens for higher granularity
CHARS = ['BLANK'] + list(" abcdefghijklmnopqrstuvwxyz'") 
CHAR_MAP = {c: i for i, c in enumerate(CHARS)}

# Dynamic discovery of all available training HDF5 files across the dataset directory
TRAIN_FILES = [str(p) for p in Path(BASE_PATH).rglob('data_train.hdf5')]

# --- 2. Advanced Transformer-based Architecture ---
# Implementing a Multi-Head Self-Attention Encoder to capture long-range temporal dependencies
class NeuralTransformerModel(nn.Module):
    def __init__(self, input_dim=512, hidden_dim=256, nhead=8, num_layers=6):
        super(NeuralTransformerModel, self).__init__()
        
        # Initial projection layer to align neural features with transformer dimensionality
        self.feature_projection = nn.Linear(input_dim, hidden_dim)
        
        # Transformer Encoder Layer with 8 attention heads for diverse feature extraction
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=hidden_dim, 
            nhead=nhead, 
            dim_feedforward=hidden_dim * 4, 
            dropout=0.2, 
            batch_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        
        # Output classifier projecting hidden states to character probability space
        self.final_classifier = nn.Linear(hidden_dim, len(CHARS))

    def forward(self, x):
        # x: [Batch, Sequence_Length, Input_Dim]
        x = self.feature_projection(x)
        x = self.transformer_encoder(x)
        # Apply Log-Softmax for CTC Loss compatibility
        return self.final_classifier(x).log_softmax(2)

# --- 3. Data Engineering and Signal Preprocessing ---
class NeuralSignalDataset(Dataset):
    def __init__(self, file_paths):
        self.samples = []
        for path in tqdm(file_paths, desc="Preprocessing Neural Data"):
            if not os.path.exists(path): continue
            with h5py.File(path, 'r') as hf:
                for key in hf.keys():
                    # Load high-dimensional neural input features
                    features = hf[key]['input_features'][()]
                    
                    # Apply Z-score Normalization to stabilize signal variance
                    features = (features - np.mean(features)) / (np.std(features) + 1e-6)
                    
                    # Extract and decode transcription labels
                    raw_text = hf[key].get('sentence', [b''])[0].decode('utf-8').lower()
                    if not raw_text: continue
                    
                    # Convert characters to indices based on CHAR_MAP
                    target_indices = [CHAR_MAP[c] for c in raw_text if c in CHAR_MAP]
                    self.samples.append((features, target_indices))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        features, target = self.samples[idx]
        return torch.tensor(features).float(), torch.tensor(target).long()

# --- 4. Optimization and Training Routine ---
def execute_training_pipeline():
    print(f"Status: Initializing Dataset with {len(TRAIN_FILES)} files...")
    dataset = NeuralSignalDataset(TRAIN_FILES)
    
    model = NeuralTransformerModel().to(DEVICE)
    
    # Utilizing AdamW optimizer for weight decay regularization
    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=0.01)
    
    # Connectionist Temporal Classification (CTC) loss for unaligned sequences
    criterion = nn.CTCLoss(blank=0, zero_infinity=True)

    print("Status: Starting Model Optimization (50 Epochs Target)...")
    model.train()
    
    for epoch in range(50):
        epoch_loss = 0
        for features, targets in dataset:
            features = features.unsqueeze(0).to(DEVICE)
            targets = targets.unsqueeze(0).to(DEVICE)
            
            optimizer.zero_grad()
            
            # Forward pass: log-probabilities shape [Seq_Len, Batch, Classes] for CTC
            output = model(features).permute(1, 0, 2)
            
            input_lengths = torch.tensor([output.size(0)], dtype=torch.long)
            target_lengths = torch.tensor([targets.size(1)], dtype=torch.long)
            
            loss = criterion(output, targets, input_lengths, target_lengths)
            loss.backward()
            optimizer.step()
            
            epoch_loss += loss.item()
            
        avg_loss = epoch_loss / len(dataset)
        print(f"Epoch [{epoch+1}/50] - Training Loss: {avg_loss:.4f}")
        
    return model

# --- 5. Inference and Submission Generation ---
def generate_submission(model):
    model.eval()
    predictions = []
    test_files = sorted(list(Path(BASE_PATH).rglob("data_test.hdf5")))
    
    print("Status: Commencing Inference on Test Partitions...")
    with torch.no_grad():
        for file_path in tqdm(test_files):
            with h5py.File(file_path, "r") as hf:
                trial_keys = sorted(hf.keys(), key=lambda k: int(k.split('_')[1]) if '_' in k else 0)
                for key in trial_keys:
                    x = torch.from_numpy(hf[key]["input_features"][()]).float().unsqueeze(0).to(DEVICE)
                    # Apply identical normalization as training
                    x = (x - x.mean()) / (x.std() + 1e-6)
                    
                    logits = model(x)
                    
                    # Greedy decoding: Select highest probability indices and collapse repeats
                    best_path = torch.argmax(logits[0], dim=-1).unique_consecutive()
                    decoded_str = "".join([CHARS[i] for i in best_path if i != 0])
                    predictions.append(decoded_str.strip() if decoded_str.strip() else "sil")

    # Construct final dataframe for Kaggle submission
    submission_df = pd.DataFrame({"id": range(len(predictions)), "text": predictions})
    submission_df.to_csv("submission.csv", index=False)
    print("Final Status: Submission file 'submission.csv' generated successfully.")

# --- System Execution ---
if __name__ == "__main__":
    optimized_model = execute_training_pipeline()
    generate_submission(optimized_model)

Status: Initializing Dataset with 45 files...


Preprocessing Neural Data:   0%|          | 0/45 [00:00<?, ?it/s]

In [ ]:
import h5py, torch, os
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm.auto import tqdm
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# --- ١. ڕێکخستنی بنەڕەتی ---
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BASE_PATH = '/kaggle/input/brain-to-text-25/t15_copyTask_neuralData/hdf5_data_final'
CHARS = ['BLANK'] + list(" abcdefghijklmnopqrstuvwxyz'") 
CHAR_MAP = {c: i for i, c in enumerate(CHARS)}

TRAIN_FILES = [str(p) for p in Path(BASE_PATH).rglob('data_train.hdf5')]

# --- ٢. مۆدێلی زۆر پێشکەوتوو (Conformer Architecture) ---
# ئەم مۆدێلە سیگناڵەکان بە وردی "سەرنج" (Attention) دەدات

class BrainConformerModel(nn.Module):
    def __init__(self, input_dim=512, hidden_dim=256, nhead=8, num_layers=6):
        super().__init__()
        # چینی سەرەتا بۆ گۆڕینی سیگناڵ بۆ فەزای Transformer
        self.embedding = nn.Linear(input_dim, hidden_dim)
        
        # چینی Transformer Encoder بۆ دۆزینەوەی پەیوەندییە ئاڵۆزەکان
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=hidden_dim, nhead=nhead, dim_feedforward=hidden_dim*4, 
            dropout=0.2, batch_first=True
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        
        # چینی کۆتایی بۆ پێشبینی پیتەکان
        self.classifier = nn.Linear(hidden_dim, len(CHARS))

    def forward(self, x):
        x = self.embedding(x)
        x = self.transformer(x)
        return self.classifier(x).log_softmax(2)

# --- ٣. ئامادەکردنی داتا (Dataset) ---
class BrainDataset(Dataset):
    def __init__(self, file_paths):
        self.data = []
        for path in tqdm(file_paths, desc="Loading Data"):
            with h5py.File(path, 'r') as hf:
                for k in hf.keys():
                    feat = hf[k]['input_features'][()]
                    # ئاساییکردنەوەی سیگناڵ (Z-score normalization) زۆر گرنگە بۆ بردنەوە
                    feat = (feat - np.mean(feat)) / (np.std(feat) + 1e-6)
                    
                    sentence = hf[k].get('sentence', [b''])[0].decode('utf-8').lower()
                    if not sentence: continue
                    target = [CHAR_MAP[c] for c in sentence if c in CHAR_MAP]
                    self.data.append((feat, target))

    def __len__(self): return len(self.data)
    def __getitem__(self, idx):
        return torch.tensor(self.data[idx][0]).float(), torch.tensor(self.data[idx][1]).long()

# --- ٤. مەشقی چڕ بۆ بردنەوە (Training) ---
def train_to_win():
    dataset = BrainDataset(TRAIN_FILES)
    model = BrainConformerModel().to(DEVICE)
    
    # بەکارهێنانی OneCycleLR بۆ ئەوەی مۆدێلەکە خێراتر و باشتر فێر بێت
    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=0.01)
    criterion = nn.CTCLoss(blank=0, zero_infinity=True)

    print("🔥 مەشقکردن بە مۆدێلی Conformer دەستی پێکرد...")
    model.train()
    for epoch in range(50): # ٥٠ خول بۆ ئەوەی مۆدێلەکە هەموو شتێک فێر بێت
        total_loss = 0
        for x, y in dataset:
            x, y = x.unsqueeze(0).to(DEVICE), y.unsqueeze(0).to(DEVICE)
            optimizer.zero_grad()
            
            log_probs = model(x).permute(1, 0, 2)
            input_len = torch.tensor([log_probs.size(0)], dtype=torch.long)
            target_len = torch.tensor([y.size(1)], dtype=torch.long)
            
            loss = criterion(log_probs, y, input_len, target_len)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        
        print(f"Epoch {epoch+1}/50 | Loss: {total_loss/len(dataset):.4f}")
    return model

# --- ٥. ناردنی ئەنجام (Submission) ---
def final_submit(model):
    model.eval()
    results = []
    test_files = sorted(list(Path(BASE_PATH).rglob("data_test.hdf5")))
    
    with torch.no_grad():
        for p in tqdm(test_files):
            with h5py.File(p, "r") as hf:
                trials = sorted(hf.keys(), key=lambda k: int(k.split('_')[1]) if '_' in k else 0)
                for t in trials:
                    x = torch.from_numpy(hf[t]["input_features"][()]).float().unsqueeze(0).to(DEVICE)
                    # Normalization بۆ تێستیش پێویستە
                    x = (x - x.mean()) / (x.std() + 1e-6)
                    
                    logits = model(x)
                    indices = torch.argmax(logits[0], dim=-1).unique_consecutive()
                    pred = "".join([CHARS[i] for i in indices if i != 0])
                    results.append(pred.strip() if pred.strip() else "sil")

    pd.DataFrame({"id": range(len(results)), "text": results}).to_csv("submission.csv", index=False)
    print("🏆 تەواو! هیوادارم ئەمە نمرەی یەکەمت بۆ بهێنێت.")

# کارپێکردن
winning_model = train_to_win()
final_submit(winning_model)